In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, setdiff, join, unique, vstack
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import WMAP5 as cosmo
import astropy.constants as const
import itertools

In [2]:
# path = 'C:\\Users\\Kenneth\\Desktop\\fyp\\datasets'
data = Table.read('datasets/mass_complete_env_cleaned.csv')

In [3]:
central = np.where(data['galaxy_type'] == 'central')
central_galaxies = data[central] # 333 unique central galaxies

In [4]:
satellite = np.where(data['galaxy_type'] == 'satellite')
satellite_galaxies = data[satellite] # 7221 unique satellite galaxies

In [5]:
# check for missing groups
missing_groups_gal_id = setdiff(central_galaxies, satellite_galaxies, keys='group_id')['id']
mask = np.logical_and.reduce([central_galaxies['id'] != missing_groups_gal_id])
central_galaxies = central_galaxies[mask] # remove independent central galaxy
len(central_galaxies) # 332 central galaxies with groups

332

In [6]:
joined_satellite_galaxies = join(satellite_galaxies, central_galaxies['id', 'group_id'], keys='group_id') # add central galaxy id column to satellite galaxies
joined_satellite_galaxies.rename_column('id_2', 'ccg')
joined_satellite_galaxies.rename_column('id_1', 'id')
central_galaxies.remove_columns(['col0', 's_cluster', 's_filament', 'galaxy_type', 'environment']) # remove unnecessary columns
joined_satellite_galaxies.remove_columns(['col0', 's_cluster', 's_filament', 'galaxy_type'])
len(satellite_galaxies)

7221

In [7]:
combined_galaxies = vstack([joined_satellite_galaxies, central_galaxies]) # 7426 total galaxies
# len(joined_satellite_galaxies)+len(central_galaxies)
gal_groups = combined_galaxies.group_by('group_id')

In [8]:
# convert photo_z to los_vel
def redshift_to_velocity(z):
    c = const.c.to('m/s')
    v = c*((1+z)**2 - 1)/((1+z)**2 + 1)
    return v

In [9]:
# calculate velocity dispersion for an array of velocities
def vel_disp(v_arr):
    return np.sum(v_arr**2)/len(v_arr)

In [1]:
# for all groups
# vel_disp_arr = np.zeros(len(gal_groups.groups))
# gal_group_avg = gal_groups.groups.aggregate(np.mean)['photo_z']

# for i, group in enumerate(gal_groups.groups):
#     gal_group_doppler_z = np.array(group['photo_z']) - gal_group_avg[i]
#     gal_group_vel = redshift_redshift_to_velocity(gal_group_doppler_z)
#     gal_group_vel_disp = vel_disp(gal_group_vel)
#     vel_disp_arr[i] = gal_group_vel_disp

In [11]:
# for first group
group1 = gal_groups.groups[0]
group_average = group1['photo_z'].mean()
doppler_z = np.array(group1['photo_z']) - group_average
group_vel = redshift_to_velocity(doppler_z)
group_vel_disp = vel_disp(group_vel)
group_vel_disp

<Quantity 7.86412569e+12 m2 / s2>

In [12]:
def separation(ccg, galaxy):
    # c1 = SkyCoord(ra=ccg[0]*u.degree, dec=ccg[1]*u.degree)
    # c2 = SkyCoord(ra=galaxy[0]*u.degree, dec=galaxy[1]*u.degree)
    sep = c1.separation(c2).to(u.rad)
    d_A = cosmo.angular_diameter_distance(z=ccg[2])
    actual_sep = (sep*d_A).to(u.Mpc, u.dimensionless_angles())
    return actual_sep

In [13]:
# for first group

photoz_arr = np.asarray(group1['photo_z']) # array of photo z
c = SkyCoord(ra=group1['ra']*u.degree, dec=group1['dec']*u.degree) # array of coordinates

pairs_idx = np.asarray(list((i,j) for ((i,_), (j,_)) in itertools.combinations(enumerate(c), 2))) # index of gals in combinations
photoz_pairs = np.take(photoz_arr, pairs_idx)
d_A = cosmo.angular_diameter_distance(z=photoz_pairs)

pairs = np.asarray(list(itertools.combinations(c,2))) # pairwise combinations
separations = np.zeros(len(pairs))*(1/u.Mpc)
for idx, pair in enumerate(pairs):
    sep = pair[0].separation(pair[1])
    actual_sep = (sep*d_A[idx, 0]).to(u.Mpc, u.dimensionless_angles())
    separations[idx] = (1/actual_sep)

separation = sum(separations)

In [14]:
# for all groups
# group_separations = np.zeros(len(gal_groups.groups))

# for i, group in enumerate(gal_groups.groups):
#     separations = 0
#     arr = np.array(list(zip(group['ra'], group['dec'], group['photo_z'])))
#     for pair in itertools.combinations(arr, 2):
#         separations += (1/separation(*pair))
#     group_separations[i] = separations

In [22]:
def group_radius(group_size, separation):
    return group_size*(group_size-1)*(1/separation).to(u.m)

In [23]:
# for all groups
# group_sizes = [len(i) for i in gal_groups.groups]
# group_radius(group_separations)

In [26]:
# for first group
group_radius = group_radius(len(group1), separation)
group_radius

<Quantity 4.05487586e+22 m>

In [27]:
def group_mass(group_vel_disp, group_radius):
    mass = (3/2)*np.pi*(group_vel_disp*group_radius)/const.G
    return mass.to(u.M_sun)

In [28]:
# first group
mass = group_mass(group_vel_disp, group_radius)

1.1322895683699952e+16